In [17]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_qa_with_sources_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
from pytubefix import YouTube
import asyncio
import torch
from faster_whisper import WhisperModel

In [18]:
load_dotenv()  # .env 파일에서 환경 변수 로드

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [19]:
def get_video_info(url):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()
    return {
        "title": yt.title,
        "audio_url": audio_stream.url if audio_stream else None
    }


In [20]:
video_url = "https://www.youtube.com/shorts/a--NSC19MXM"
video_info = get_video_info(video_url)
print(f"Video Title: {video_info['title']}")

Video Title: 가장 쉬운 까르보나라


In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

In [22]:
whisper_model = WhisperModel("large-v3", device=device, compute_type=compute_type)

def transcribe_audio(audio_url):
    segments, info = whisper_model.transcribe(audio_url)
    transcript = [{"text": segment.text, "start": segment.start, "end": segment.end} for segment in segments]
    return {"script": transcript, "language": info.language}

In [23]:
transcript = transcribe_audio(video_info['audio_url'])
print(f"Transcript Language: {transcript['language']}")
print(f"First few lines of transcript: {transcript['script'][:3]}")

transcribe.py       :324  2024-10-12 11:30:14,287 Processing audio with duration 00:59.118
transcribe.py       :425  2024-10-12 11:30:19,512 Detected language 'ko' with probability 1.00


Transcript Language: ko
First few lines of transcript: [{'text': ' 한 남자가 베이컨을 가져오는데요', 'start': 0.0, 'end': 10.46}, {'text': ' 갑자기 계란을 가져와 깨부수기 시작합니다', 'start': 10.46, 'end': 12.74}, {'text': ' 노른자를 건져내 따로 담아줍니다', 'start': 12.74, 'end': 14.74}]


In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
summary_prompt = hub.pull("teddynote/summary-stuff-documents-korean")
llm = ChatOpenAI(
            model_name="gpt-4o-mini",
            temperature=0.7,
            streaming=True,
        )

/var/folders/c4/0f7nfvt16ln8630csjtkk_1w0000gn/T/ipykernel_41602/2953908480.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [25]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [26]:
from langchain_community.document_loaders import JSONLoader, TextLoader
docs = TextLoader("script.txt").load()

In [27]:
docs

[Document(metadata={'source': 'script.txt'}, page_content="In the last chapter, you and I started to step through the internal workings of a transformer.\nThis is one of the key pieces of technology inside large language models, and a lot of\nother tools in the modern wave of AI.\nIt first hit the scene in a now-famous 2017 paper called Attention is All You Need, and\nin this chapter, you and I will dig into what this attention mechanism is, visualizing how\nit processes data.\nAs a quick recap, here's the important context I want you to have in mind.\nThe goal of the model that you and I are studying is to take in a piece of text and predict\nwhat word comes next.\nThe input text is broken up into little pieces that we call tokens, and these are very often\nwords or pieces of words, but just to make the examples in this video easier for you\nand me to think about, let's simplify by pretending that tokens are always just words.\nThe first step in a transformer is to associate each toke

In [28]:
from langchain_core.documents import Document
document = [Document(page_content="\n".join([t["text"] for t in transcript["script"]]))]

In [29]:
summary_chain = create_stuff_documents_chain(llm,summary_prompt)
result = await summary_chain.ainvoke({"context": document})

_client.py          :1786 2024-10-12 11:31:22,301 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [30]:
result

'- 🥓 한 남자가 베이컨을 가져옴.  \n- 🥚 계란을 깨고 노른자를 따로 담음.  \n- 🥄 흰자는 생으로 먹고, 소금을 넣음.  \n- 🌳 나무젓가락을 사용함.  \n- 🍝 파스타를 넣고 재료를 섞음.  \n- 🧂 후추와 그라라빠다노를 뿌림.  \n- 🔥 베이컨을 구워 계란치즈 소스를 섞음.  \n- 🍽️ 까르보나라를 예쁘게 담아 완성함.  \n- 🤤 저도 한번 꼭 먹어보고 싶네요.  '

In [31]:
embeddings = OpenAIEmbeddings()

/var/folders/c4/0f7nfvt16ln8630csjtkk_1w0000gn/T/ipykernel_41602/2497576997.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [32]:
result.strip().split("\n")

['- 🥓 한 남자가 베이컨을 가져옴.  ',
 '- 🥚 계란을 깨고 노른자를 따로 담음.  ',
 '- 🥄 흰자는 생으로 먹고, 소금을 넣음.  ',
 '- 🌳 나무젓가락을 사용함.  ',
 '- 🍝 파스타를 넣고 재료를 섞음.  ',
 '- 🧂 후추와 그라라빠다노를 뿌림.  ',
 '- 🔥 베이컨을 구워 계란치즈 소스를 섞음.  ',
 '- 🍽️ 까르보나라를 예쁘게 담아 완성함.  ',
 '- 🤤 저도 한번 꼭 먹어보고 싶네요.']

In [33]:
documents = text_splitter.create_documents([t["text"] for t in transcript["script"]])
for doc in documents:
    doc.page_content += "\n" + summary.strip()

NameError: name 'summary' is not defined

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("RUNPOD_API_KEY")
you_url = "https://youtube.com/shorts/a--NSC19MXM?si=yiun-HK_7wX1sNvL"

In [2]:
import requests
import os
from dotenv import load_dotenv


# RunPod RUNSYNC 엔드포인트 URL
url = "https://api.runpod.ai/v2/uq96boxkzy99ev/runsync"

# FastAPI의 /hello 엔드포인트로 요청하기 위한 데이터 (내부적으로 사용할 파라미터 설정)
body = {"input":{
    "api":{
        "method":"POST",
        "endpoint":"/ping",
    },
    "payload":{},
}}
# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# RUNSYNC 요청 보내기
response = requests.post(url, json=body, headers=headers)

# 응답 확인
if response.status_code == 200:
    print("Response:", response.json())
else:
    print(f"Error {response.status_code}: {response.text}")


Response: {'id': 'sync-26147017-ff6e-408d-9622-80c484868c42-e1', 'status': 'IN_QUEUE'}


In [7]:
import requests

# 작업 ID (작업 완료된 job ID)
job_id = response.json()['id']

# RunPod API STATUS 엔드포인트 URL
status_url = f"https://api.runpod.ai/v2/wm1xrz07all039/status/{job_id}"


# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}"
}

# 작업 상태 및 결과 확인 요청 보내기
response = requests.get(status_url, headers=headers)

# 응답 확인
if response.status_code == 200:
    job_result = response.json()
    if job_result.get("status") == "COMPLETED":
        print("Job Completed! Result:", job_result.get("output"))
    else:
        print(f"Job Status: {job_result.get('status')}")
else:
    print(f"Error {response.status_code}: {response.text}")


Job Status: IN_QUEUE


In [27]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.getenv("RUNPOD_API_KEY")
endpoint_id = os.getenv("RUNPOD_ENDPOINT_ID")

# RunPod RUNSYNC 엔드포인트 URL
url = f"https://api.runpod.ai/v2/{endpoint_id}/runsync"
you_url = "https://youtu.be/omEk2BNDt1I?si=xjtbYANtlux5CTfB"


# FastAPI의 /hello 엔드포인트로 요청하기 위한 데이터
payload = {
    "input": {
        "endpoint": "/get_title_hash",
        "method": "GET",
        # "headers": {"x-session-id": "1234asdf"},
        "params": {"url": you_url},
    }
}

# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# RUNSYNC 요청 보내기
response = requests.post(url, json=payload, headers=headers)

# 응답 확인
if response.status_code == 200:
    print("Response:", response.json())
else:
    print(f"Error {response.status_code}: {response.text}")


Response: {'delayTime': 6071, 'executionTime': 3825, 'id': 'sync-c9bea7a9-08ea-447d-bd62-e481515985b4-e1', 'output': {'hashtags': '#파뿌리 #생일 #친구 #예능 #노랭이', 'title': '수제 김밥 30줄로 생일 파티합니다!! 역대급 선물 언박싱까지!!!!!!!'}, 'status': 'COMPLETED', 'workerId': 'vto3bvdf9z7v0a'}


In [28]:
# import requests
# import os
# from dotenv import load_dotenv

# # Load environment variables
# load_dotenv()
# api_key = os.getenv("RUNPOD_API_KEY")
# endpoint_id = os.getenv("RUNPOD_ENDPOINT_ID")

# # RunPod RUNSYNC 엔드포인트 URL
# url = f"https://api.runpod.ai/v2/{endpoint_id}/runsync"

# # FastAPI의 /hello 엔드포인트로 요청하기 위한 데이터
# payload = {
#     "input": {
#         "endpoint": "/get_script_summary",
#         "method": "GET",
#         "headers": {"x-session-id": "1234asdf"},
#         "params": {"url": you_url},
#     }
# }

# # 요청 헤더에 API 키 추가
# headers = {
#     "Authorization": f"Bearer {api_key}",
#     "Content-Type": "application/json"
# }

# # RUNSYNC 요청 보내기
# response = requests.post(url, json=payload, headers=headers)

# # 응답 확인
# if response.status_code == 200:
#     print("Response:", response.json())
# else:
#     print(f"Error {response.status_code}: {response.text}")


In [33]:
import requests
import os
import time
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.getenv("RUNPOD_API_KEY")
endpoint_id = os.getenv("RUNPOD_ENDPOINT_ID")

# RunPod RUNSYNC 엔드포인트 URL
url = f"https://api.runpod.ai/v2/{endpoint_id}/run"

# FastAPI의 /hello 엔드포인트로 요청하기 위한 데이터
payload = {
    "input": {
        "endpoint": "/get_script_summary",
        "method": "GET",
        "headers": {"x-session-id": "1234asdf"},
        "params": {"url": you_url},
    }
}

# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# RUNSYNC 요청 보내기
response = requests.post(url, json=payload, headers=headers)

# 응답 확인
if response.status_code == 200:
    result = response.json()
    print("Initial Response:", result)
    
    if result.get('status') in ['IN_PROGRESS',"IN_QUEUE"]:
        job_id = result.get('id')
        status_url = f"https://api.runpod.ai/v2/{endpoint_id}/status/{job_id}"
        
        while True:
            status_response = requests.get(status_url, headers=headers)
            if status_response.status_code == 200:
                status_data = status_response.json()
                print(f"Current status: {status_data.get('status')}")
                
                if status_data.get('status') == 'COMPLETED':
                    print(f"결과값:{status_data}")
                    result_url = f"https://api.runpod.ai/v2/{endpoint_id}/result/{job_id}"
                    result_response = requests.get(result_url, headers=headers)
                    
                    if result_response.status_code == 200:
                        final_result = result_response.json()
                        print("Final Result:", final_result)
                        break
                    else:
                        print(f"Error fetching results: {result_response.status_code}")
                        print(f"Error message: {result_response.text}")
                        break
                elif status_data.get('status') == 'FAILED':
                    print("Job failed")
                    break
            else:
                print(f"Error checking status: {status_response.status_code}")
                print(f"Error message: {status_response.text}")
                break
            
            time.sleep(5)  # 5초 대기 후 다시 상태 확인
    else:
        print("Job completed immediately")
        print("Final Result:", result)
else:
    print(f"Error {response.status_code}: {response.text}")

Initial Response: {'id': 'c2cb7372-07b6-4146-b2a4-7c8ad4e0eb34-e1', 'status': 'IN_QUEUE'}
Current status: IN_QUEUE
Current status: IN_QUEUE
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_PROGRESS
Current status: IN_P

In [24]:
import requests
import os
from dotenv import load_dotenv
import time

# Load environment variables
load_dotenv()
api_key = os.getenv("RUNPOD_API_KEY")
endpoint_id = os.getenv("RUNPOD_ENDPOINT_ID")

# RunPod RUNSYNC 엔드포인트 URL
url = f"https://api.runpod.ai/v2/{endpoint_id}/status/{job_id}"
url2 = f"https://api.runpod.ai/v2/{endpoint_id}/result/{job_id}"

# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

while True:
    # RUNSYNC 요청 보내기
    response = requests.get(url,headers=headers)

    # 응답 확인
    if response.status_code == 200:
        if response.json().get("status") == "COMPLETED":
            response = requests.get(url2,headers=headers)
            break
        else:
            print(f"Job status: {response.json().get('status')}")
    else:
        print(f"Error {response.status_code}: {response.text}")
    time.sleep(5)


Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS


In [26]:
response.text

'404 page not found'

In [16]:
import json
RUNPOD_API_URL = f"https://api.runpod.ai/v2/{endpoint_id}/runsync"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}
payload = {
    "input": {
        "endpoint": "/rag_stream_chat",
        "method": "POST",
        "headers": {"x-session-id": "1234asdf"},
        "params": {"prompt": "영상의 주제가 뭔가요?"},
    }
}

response = requests.post(
    RUNPOD_API_URL, headers=headers, json=payload, stream=True
)

# for chunk in response.iter_content(chunk_size=None):
#     if chunk:
#         chunk_data = chunk.decode("utf-8").strip()
#         if chunk_data.startswith("data: "):
#             chunk_content = chunk_data[6:]
#             if chunk_content == "[DONE]":
#                 break
#             try:
#                 content = json.loads(chunk_content)
#                 print("Stream content:", content)
#             except json.JSONDecodeError:
#                 print("Invalid JSON:", chunk_content)


In [28]:
answer = ""
for chunk in response.json().get("output"):
    answer += chunk.get("content")
    print(answer)


영상
영상의
영상의 주
영상의 주제
영상의 주제는
영상의 주제는 까
영상의 주제는 까르
영상의 주제는 까르보
영상의 주제는 까르보나라
영상의 주제는 까르보나라 요
영상의 주제는 까르보나라 요리
영상의 주제는 까르보나라 요리 과정
영상의 주제는 까르보나라 요리 과정에
영상의 주제는 까르보나라 요리 과정에 대한
영상의 주제는 까르보나라 요리 과정에 대한 내용
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다.
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨,
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨, 계
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨, 계란
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨, 계란,
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨, 계란, 면
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨, 계란, 면수
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨, 계란, 면수 등을
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨, 계란, 면수 등을 사용
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨, 계란, 면수 등을 사용하여
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨, 계란, 면수 등을 사용하여 맛
영상의 주제는 까르보나라 요리 과정에 대한 내용입니다. 영상에서는 베이컨, 계란, 면수 등

In [3]:
with open("script.txt","r") as f:
    lines = f.readlines()

In [5]:
context = "".join(lines)

In [7]:
print(context)

In the last chapter, you and I started to step through the internal workings of a transformer.
This is one of the key pieces of technology inside large language models, and a lot of
other tools in the modern wave of AI.
It first hit the scene in a now-famous 2017 paper called Attention is All You Need, and
in this chapter, you and I will dig into what this attention mechanism is, visualizing how
it processes data.
As a quick recap, here's the important context I want you to have in mind.
The goal of the model that you and I are studying is to take in a piece of text and predict
what word comes next.
The input text is broken up into little pieces that we call tokens, and these are very often
words or pieces of words, but just to make the examples in this video easier for you
and me to think about, let's simplify by pretending that tokens are always just words.
The first step in a transformer is to associate each token with a high-dimensional vector,
what we call its embedding.
Now the m

In [8]:
import tiktoken

# gpt-3.5-turbo 모델의 토큰 수 계산
def calculate_tokens_gpt35(text):
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    tokens = encoding.encode(text)
    return len(tokens)

# gpt-4 모델의 토큰 수 계산
def calculate_tokens_gpt4(text):
    encoding = tiktoken.encoding_for_model("gpt-4")
    tokens = encoding.encode(text)
    return len(tokens)

# gpt-4 모델의 토큰 수 계산
def calculate_tokens_gpt4o_mini(text):
    encoding = tiktoken.encoding_for_model("gpt-4o-mini")
    tokens = encoding.encode(text)
    return len(tokens)



# gpt-3.5-turbo의 토큰 수 계산
tokens_gpt35 = calculate_tokens_gpt35(context)
print(f"GPT-3.5-turbo 토큰 수: {tokens_gpt35}")

# gpt-4의 토큰 수 계산
tokens_gpt4 = calculate_tokens_gpt4(context)
print(f"GPT-4 토큰 수: {tokens_gpt4}")

# gpt-4의 토큰 수 계산
tokens_gpt4o = calculate_tokens_gpt4o_mini(context)
print(f"GPT-4o-mini 토큰 수: {tokens_gpt4o}")


GPT-3.5-turbo 토큰 수: 5831
GPT-4 토큰 수: 5831
GPT-4o-mini 토큰 수: 5729


In [ ]:
# gpt-4 모델의 토큰 수 계산
def calculate_tokens_gpt4(text):
    encoding = tiktoken.encoding_for_model("gpt-4")
    tokens = encoding.encode(text)
    return len(tokens)
